# Gridding visibilities.

In this first tutorial, I'll show how to access the visibilities from a CASA Measurement Set and how to grid them. This is the only step needed to prepare the data for the inference. We'll take for example the RULup protoplanetary disk  